# Parsing text files and text preprocessing of Covid19 Tweets

## Table of Content
1. [Parsing Text Files](#1)
2. [Text Preprocessing](#2)

## 1. Parsing Text Files <a class="anchor" id="1"></a>

## 2. Text Preprocessing <a class="anchor" id="2"></a>